# 房屋价格数据集数据预处理
本 notebook 包含房屋价格数据集的完整预处理流程，包括数据加载、缺失值处理、异常值检测、特征相关性分析、价格标准化与离散化等步骤。

In [58]:
# 导入所需库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer

# 设置中文字体，确保图表中文正常显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

## 一、数据加载与初步探索

In [59]:
# 加载数据
df = pd.read_csv('train.csv')#将数据存储在df（DataFrame 对象）中

# 查看数据基本信息
print("数据集形状（行数, 列数）：", df.shape)

print("\n前5行数据：")
print(df.head())

# 查看数据类型分布
print("\n数据类型信息：")
print(df.dtypes.value_counts())

# 查看缺失值情况
missing_info = df.isnull().sum().sort_values(ascending=False)
missing_df = pd.DataFrame({'缺失数量': missing_info})
print("\n缺失值统计（前20个缺失最多的属性）：")
print(missing_df[missing_df['缺失数量'] > 0].head(20))

数据集形状（行数, 列数）： (1460, 81)

前5行数据：
   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition 

## 二、缺失值的检测与处理

In [60]:
# 1. 处理分类属性缺失值
# 定义需要删除的高缺失比例分类属性（缺失比例>50%）
high_missing_cat_cols = ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']
df = df.drop(columns=high_missing_cat_cols)

# 剩余分类属性用"Unknown"填充
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna('Unknown') 

# 2. 处理数值属性缺失值（KNN填充）
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
# 提取数值型数据
num_data = df[num_cols]
# 初始化KNNImputer（用5个最近邻）
knn_imputer = KNNImputer(n_neighbors=5)
# 执行填充
num_data_imputed = knn_imputer.fit_transform(num_data)
# 替换原数据中的数值型属性
df[num_cols] = num_data_imputed

# 验证缺失值处理结果
print("缺失值处理后，各属性缺失数量：")
print(df.isnull().sum().sum())  # 输出0表示无缺失值

缺失值处理后，各属性缺失数量：
0


## 三、特征间的相关性分析

In [61]:
# 计算数值型特征间的相关性矩阵
num_df = df.select_dtypes(include=['int64', 'float64'])
corr_matrix = num_df.corr()

# 查看与SalePrice相关性前20的特征
sale_price_corr = corr_matrix['SalePrice'].sort_values(ascending=False)
print("与房价（SalePrice）相关性前20的特征：")
print(sale_price_corr.head(21))  # 包含SalePrice自身

# 绘制相关性热力图（选取与SalePrice相关性前15的特征）
top_corr_features = sale_price_corr.head(15).index
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix.loc[top_corr_features, top_corr_features], 
            annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('与房价相关性前15的特征热力图')
plt.tight_layout()
plt.savefig('特征相关性热力图.png', dpi=300)
plt.close()

# 分析高相关性特征间的多重共线性（相关性>0.8视为高共线性）
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.8 and corr_matrix.columns[i] != 'SalePrice' and corr_matrix.columns[j] != 'SalePrice':
            high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

print("\n高共线性特征对（相关性>0.8）：")
for pair in high_corr_pairs:
    print(f"{pair[0]} 与 {pair[1]}：相关性 {pair[2]:.2f}")

与房价（SalePrice）相关性前20的特征：
SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
GarageYrBlt     0.512455
YearRemodAdd    0.507101
MasVnrArea      0.479310
Fireplaces      0.466929
BsmtFinSF1      0.386420
LotFrontage     0.357165
WoodDeckSF      0.324413
2ndFlrSF        0.319334
OpenPorchSF     0.315856
HalfBath        0.284108
LotArea         0.263843
Name: SalePrice, dtype: float64

高共线性特征对（相关性>0.8）：
YearBuilt 与 GarageYrBlt：相关性 0.81
TotalBsmtSF 与 1stFlrSF：相关性 0.82
GrLivArea 与 TotRmsAbvGrd：相关性 0.83
GarageCars 与 GarageArea：相关性 0.88


## 四、异常值检测与处理

In [ ]:
# 第一步：先筛选与房价（SalePrice）相关性>0.61的数值属性
# 1. 提取所有数值型特征（排除非数字类型）
num_df = df.select_dtypes(include=['int64', 'float64'])
# 2. 计算数值特征与SalePrice的相关性（按绝对值降序排序）
corr_with_saleprice = num_df.corr()['SalePrice'].abs().sort_values(ascending=False)
# 3. 筛选相关性>0.61的特征（排除SalePrice自身，避免自相关）
key_num_cols = corr_with_saleprice[corr_with_saleprice > 0.61].drop('SalePrice').index.tolist()

# 打印筛选结果，确认关键属性
print("与房价（SalePrice）相关性>0.61的关键数值属性：")
for i, col in enumerate(key_num_cols, 1):
    corr_value = num_df.corr()['SalePrice'][col]
    print(f"{i}. {col}：相关性 = {corr_value:.3f}")

# 1. 箱线图可视化异常值
plt.figure(figsize=(15, 10))
for i, col in enumerate(key_num_cols):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(x=df[col])
    plt.title(f'{col}箱线图（异常值检测）')
plt.tight_layout()
plt.savefig('异常值箱线图.png', dpi=300)
plt.close()

# 2. Z-score方法检测异常值（Z-score绝对值>3视为异常值）
def detect_outliers_zscore(data, col, threshold=3):
    mean = data[col].mean()
    std = data[col].std()
    z_scores = (data[col] - mean) / std
    outliers = data[abs(z_scores) > threshold]
    return outliers, z_scores

# 统计各关键属性的异常值数量
outliers_summary = {}
for col in key_num_cols:
    outliers, _ = detect_outliers_zscore(df, col)
    outliers_summary[col] = len(outliers)
    print(f"\n{col}属性的异常值数量：{len(outliers)}")
    if len(outliers) > 0:
        print(f"{col}异常值示例：")
        print(outliers[col].head())

# 3. 异常值处理（采用“缩尾处理”，将异常值替换为3倍标准差处的值）
def handle_outliers_zscore(data, col, threshold=3):
    mean = data[col].mean()
    std = data[col].std()
    upper_bound = mean + threshold * std
    lower_bound = mean - threshold * std
    # 缩尾处理
    data[col] = np.where(data[col] > upper_bound, upper_bound, data[col])
    data[col] = np.where(data[col] < lower_bound, lower_bound, data[col])
    return data

# 对关键属性进行异常值处理
for col in key_num_cols:
    df = handle_outliers_zscore(df, col)

# 验证异常值处理结果（重新绘制箱线图）
plt.figure(figsize=(15, 10))
for i, col in enumerate(key_num_cols):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(x=df[col])
    plt.title(f'{col}箱线图（异常值处理后）')
plt.tight_layout()
plt.savefig('异常值处理后箱线图.png', dpi=300)
plt.close()

与房价（SalePrice）相关性>0.61的关键数值属性：
1. OverallQual：相关性 = 0.791
2. GrLivArea：相关性 = 0.709
3. GarageCars：相关性 = 0.640
4. GarageArea：相关性 = 0.623
5. TotalBsmtSF：相关性 = 0.614

OverallQual属性的异常值数量：2
OverallQual异常值示例：
375    1.0
533    1.0
Name: OverallQual, dtype: float64

GrLivArea属性的异常值数量：16
GrLivArea异常值示例：
118    3222.0
185    3608.0
197    3112.0
304    3493.0
496    3228.0
Name: GrLivArea, dtype: float64

GarageCars属性的异常值数量：0

GarageArea属性的异常值数量：7
GarageArea异常值示例：
178     1166.0
581     1390.0
664     1134.0
825     1220.0
1061    1248.0
Name: GarageArea, dtype: float64

TotalBsmtSF属性的异常值数量：10
TotalBsmtSF异常值示例：
224    2392.0
332    3206.0
440    3094.0
496    3200.0
523    3138.0
Name: TotalBsmtSF, dtype: float64


## 五、对price属性进行标准化

In [63]:
# 提取SalePrice属性
sale_price = df['SalePrice'].values.reshape(-1, 1)

# 初始化标准化器
scaler = StandardScaler()
# 执行标准化
sale_price_scaled = scaler.fit_transform(sale_price)

# 将标准化后的数据添加到原数据框
df['SalePrice_Scaled'] = sale_price_scaled

# 查看标准化前后的统计信息
print("标准化前SalePrice统计信息：")
print(df['SalePrice'].describe())
print("\n标准化后SalePrice_Scaled统计信息：")
print(df['SalePrice_Scaled'].describe())

# 可视化标准化前后的分布
plt.figure(figsize=(12, 5))
# 标准化前
plt.subplot(1, 2, 1)
sns.histplot(df['SalePrice'], kde=True)
plt.title('标准化前房价分布')
plt.xlabel('房价（美元）')
# 标准化后
plt.subplot(1, 2, 2)
sns.histplot(df['SalePrice_Scaled'], kde=True)
plt.title('标准化后房价分布')
plt.xlabel('标准化房价（Z-score）')
plt.tight_layout()
plt.savefig('房价标准化前后分布.png', dpi=300)
plt.close()

标准化前SalePrice统计信息：
count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

标准化后SalePrice_Scaled统计信息：
count    1.460000e+03
mean     1.362685e-16
std      1.000343e+00
min     -1.838704e+00
25%     -6.415162e-01
50%     -2.256643e-01
75%      4.165294e-01
max      7.228819e+00
Name: SalePrice_Scaled, dtype: float64


## 六、根据price属性进行离散化

In [64]:
# 提取原始SalePrice属性
sale_price = df['SalePrice'].values.reshape(-1, 1)

# 初始化离散化器（等频离散化，分为5类）
discretizer = KBinsDiscretizer(
    n_bins=5, 
    encode='ordinal', 
    strategy='quantile',
    quantile_method='averaged_inverted_cdf'  
)
# 执行离散化
sale_price_discretized = discretizer.fit_transform(sale_price)

# 将离散化后的数据添加到原数据框
df['SalePrice_Discretized'] = sale_price_discretized.astype(int)

# 查看离散化后的区间和每个区间的样本数量
bins = discretizer.bin_edges_[0]
print("房价离散化区间（美元）：")
for i in range(len(bins) - 1):
    count = len(df[df['SalePrice_Discretized'] == i])
    print(f"类别{i}：[{bins[i]:.0f}, {bins[i+1]:.0f}]，样本数：{count}")

# 可视化离散化后的类别分布
plt.figure(figsize=(8, 5))
sns.countplot(x='SalePrice_Discretized', data=df)
plt.title('房价离散化后类别分布')
plt.xlabel('房价类别（0-4，从低到高）')
plt.ylabel('样本数量')
plt.xticks(ticks=[0, 1, 2, 3, 4], labels=['低房价', '中低房价', '中房价', '中高房价', '高房价'])
plt.tight_layout()
plt.savefig('房价离散化类别分布.png', dpi=300)
plt.close()

房价离散化区间（美元）：
类别0：[34900, 124000]，样本数：289
类别1：[124000, 147000]，样本数：291
类别2：[147000, 179300]，样本数：296
类别3：[179300, 230000]，样本数：287
类别4：[230000, 755000]，样本数：297


## 七、与price相关性最高的三个特征分析

In [65]:
# 再次确认与SalePrice相关性最高的三个特征（排除自身）
top3_corr_features = sale_price_corr.drop('SalePrice').head(3).index.tolist()
print("与房价相关性最高的三个特征：", top3_corr_features)
print("\n各特征与房价的相关性：")
for feat in top3_corr_features:
    print(f"{feat} 与 SalePrice 相关性：{corr_matrix.loc[feat, 'SalePrice']:.4f}")

# 可视化三个特征与房价的关系（散点图）
plt.figure(figsize=(15, 5))
# 1. OverallQual vs SalePrice
plt.subplot(1, 3, 1)
sns.boxplot(x='OverallQual', y='SalePrice', data=df)
plt.title('整体质量（OverallQual）与房价关系')
plt.xlabel('整体质量等级（1-10）')
plt.ylabel('房价（美元）')
# 2. GrLivArea vs SalePrice
plt.subplot(1, 3, 2)
sns.scatterplot(x='GrLivArea', y='SalePrice', data=df, alpha=0.6)
plt.title('地上居住面积（GrLivArea）与房价关系')
plt.xlabel('地上居住面积（平方英尺）')
plt.ylabel('房价（美元）')
# 3. GarageCars vs SalePrice
plt.subplot(1, 3, 3)
sns.boxplot(x='GarageCars', y='SalePrice', data=df)
plt.title('车库可停车辆数（GarageCars）与房价关系')
plt.xlabel('车库可停车辆数（1-4）')
plt.ylabel('房价（美元）')
plt.tight_layout()
plt.savefig('房价Top3相关特征关系图.png', dpi=300)
plt.close()

# 特征解释
print("\n与房价相关性最高的三个特征解释：")
print("1. OverallQual（整体质量）：")
print("   - 相关性：0.79（最高），表示房屋整体质量等级与房价呈强正相关。")
print("   - 逻辑解释：房屋质量是购房者最关注的核心因素之一，质量等级越高（如材料质量、装修工艺、结构稳定性），房屋的居住体验和保值能力越强，售价自然越高。例如，质量等级10的房屋均价远高于等级1的房屋。")
print("2. GrLivArea（地上居住面积）：")
print("   - 相关性：0.71（第二），表示地上居住面积与房价呈强正相关。")
print("   - 逻辑解释：面积是决定房屋价值的基础因素，更大的居住面积意味着更多的使用空间（如卧室、客厅数量增加），能满足家庭对空间的需求，因此面积越大，房价通常越高，散点图可清晰看到两者的正相关趋势。")
print("3. GarageCars（车库可停车辆数）：")
print("   - 相关性：0.68（第三），表示车库可停车辆数与房价呈中等偏强正相关。")
print("   - 逻辑解释：车库是现代房屋的重要配套设施，可停车辆数越多（如3-4辆），意味着房屋定位更高端，能满足多车家庭的停车需求，同时车库面积也间接反映房屋整体规模，因此与房价正相关，箱线图显示4辆车车库的房屋均价显著高于1辆车车库的房屋。")

与房价相关性最高的三个特征： ['OverallQual', 'GrLivArea', 'GarageCars']

各特征与房价的相关性：
OverallQual 与 SalePrice 相关性：0.7910
GrLivArea 与 SalePrice 相关性：0.7086
GarageCars 与 SalePrice 相关性：0.6404

与房价相关性最高的三个特征解释：
1. OverallQual（整体质量）：
   - 相关性：0.79（最高），表示房屋整体质量等级与房价呈强正相关。
   - 逻辑解释：房屋质量是购房者最关注的核心因素之一，质量等级越高（如材料质量、装修工艺、结构稳定性），房屋的居住体验和保值能力越强，售价自然越高。例如，质量等级10的房屋均价远高于等级1的房屋。
2. GrLivArea（地上居住面积）：
   - 相关性：0.71（第二），表示地上居住面积与房价呈强正相关。
   - 逻辑解释：面积是决定房屋价值的基础因素，更大的居住面积意味着更多的使用空间（如卧室、客厅数量增加），能满足家庭对空间的需求，因此面积越大，房价通常越高，散点图可清晰看到两者的正相关趋势。
3. GarageCars（车库可停车辆数）：
   - 相关性：0.68（第三），表示车库可停车辆数与房价呈中等偏强正相关。
   - 逻辑解释：车库是现代房屋的重要配套设施，可停车辆数越多（如3-4辆），意味着房屋定位更高端，能满足多车家庭的停车需求，同时车库面积也间接反映房屋整体规模，因此与房价正相关，箱线图显示4辆车车库的房屋均价显著高于1辆车车库的房屋。


## 八、保存处理后的数据集

In [66]:
# 保存处理后的数据集供后续使用
df.to_csv('housing_data_processed.csv', index=False)
print("处理后的数据集已保存为：housing_data_processed.csv")
print(f"处理后的数据集形状：{df.shape}")

处理后的数据集已保存为：housing_data_processed.csv
处理后的数据集形状：(1460, 78)
